In [16]:
import pandas as pd

from Dates import bump_date
from PairsTrading import suitable_pairs
from Plots import plot_spread_bollinger, plot_spreads_dislocations

In [17]:
def plot_pairs_dashboard(
    type_of_spreads: pd.DataFrame,
    long_tenor: str,
    short_tenor: str,
    window: int = 63,
    stationarity_lookback : str = '2Y',
    confidence: float = 0.75,
    sigma_factor: float = 2.0,
):
    """
    Combine the summary table (res), dislocation table, and Bollinger time-series into one Plotly figure.
    """
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    def get_rates_data():
        swap = pd.read_excel('data/MX_Rates.xlsx', index_col=0, sheet_name='Swap', parse_dates=True)
        swap.index = [d.date() for d in swap.index]
        bonds = pd.read_excel('data/MX_Rates.xlsx', index_col=0, sheet_name='Bonds', parse_dates=True)
        bonds.index = [d.date() for d in bonds.index]
        tenors = swap.columns.intersection(bonds.columns)
        ASW = (bonds[tenors] - swap[tenors])*100
        ASW.dropna(inplace = True, how = 'all')
        return bonds, swap, ASW
    bonds, swap, ASW = get_rates_data()
    if type_of_spreads.upper() == 'ASW':
        data = ASW
    elif type_of_spreads.upper() == 'BOND':
        data = bonds
    elif type_of_spreads.upper() == 'SWAP':
        data = swap
    else:
        raise Exception(f'Type of spread ({type_of_spreads}) invalid. Only available for bonds, swaps and ASW.')
    # Filter data
    end = data.index[-1]
    start = bump_date(end, '-' + stationarity_lookback)
    cointegration_data = data[(data.index > start) & (data.index <= end) ]
    # Run cointegration test
    res = suitable_pairs(cointegration_data)
    res = res[(res['Cointegrated']) & (res['Half-Life (Days)'] <= 10)].sort_values('Half-Life (Days)')
    # Clean up the res table for display
    res_display = res.copy()
    res_display = res_display.reset_index()
    res_display = res_display.rename(columns={res_display.columns[0]: "Pair"})
    for col in res_display.select_dtypes(include=["float", "int"]).columns:
        res_display[col] = res_display[col].round(4)

    res_table = go.Table(
        header=dict(
            values=list(res_display.columns),
            fill_color="lightgray",
            align="center",
            font=dict(color="black", size=12, family="Arial Black"),
        ),
        cells=dict(
            values=[res_display[c].astype(str).tolist() for c in res_display.columns],
            align="center",
            height=26,
        ),
    )

    dislocations_fig = plot_spreads_dislocations(data, window=window, confidence=confidence)
    bollinger_fig = plot_spread_bollinger(
        data, long_tenor=long_tenor, short_tenor=short_tenor, window=window, sigma_factor=sigma_factor
    )

    fig = make_subplots(
        rows=2,
        cols=2,
        specs=[[{"type": "table"}, {"type": "table"}], [{"type": "xy", "colspan": 2}, None]],
        column_widths=[0.45, 0.55],
        row_heights=[0.45, 0.55],
        horizontal_spacing=0.08,
        vertical_spacing=0.08,
        subplot_titles=(
            "Pairs Stationarity",
            dislocations_fig.layout.title.text or "Spread Z-Scores",
            bollinger_fig.layout.title.text or "Spread Bollinger Bands",
        ),
    )

    fig.add_trace(res_table, row=1, col=1)
    for trace in dislocations_fig.data:
        fig.add_trace(trace, row=1, col=2)
    for trace in bollinger_fig.data:
        fig.add_trace(trace, row=2, col=1)

    fig.update_layout(
        title_text="Rates Pairs",
        height=950,
        width=1200,
        template="plotly_white",
        showlegend=True,
    )

    fig.update_xaxes(title_text=bollinger_fig.layout.xaxis.title.text or "Date", row=2, col=1)
    fig.update_yaxes(title_text=bollinger_fig.layout.yaxis.title.text or "Spread Level", row=2, col=1)

    return fig

In [26]:
plot_pairs_dashboard('ASW', '5Y', '4Y', window=63, stationarity_lookback='2Y', confidence=0.7, sigma_factor=1.0)